# JAM FITTER

In [1]:
import sys,os
from fitlab.resman import RESMAN
from fitlab.mcsamp import MCSAMP
from tools.config import load_config,conf
from tools.tools import load, save,checkdir
import pylab as py
import numpy as np
import pandas as pd
import copy
def lprint(msg):
    sys.stdout.write('\r')
    sys.stdout.write('%s' %msg)
    sys.stdout.flush()
%matplotlib inline

## generate excel file for the new data

Lets open one the excel files in the database

To do the simulation one needs to create an xlsx file with columns as shown above. You can prepare an xlsx file outside of the program and save the file in the same place as the notebook

For the purpose of illustration, we will create the xlsx file from here

## data sets 

In [2]:
conf['datasets']={}

### HERMES data

In [3]:
conf['datasets']['sidis']={}
conf['datasets']['sidis']['filters']={0:{'idx':None,'filter':None}}
 
conf['datasets']['sidis']['filters'][0]['idx']=[9000]
conf['datasets']['sidis']['filters'][0]['filter']="z<0.6 and Q2>1.69 and pT>0.2 and pT<0.9"
conf['datasets']['sidis']['xlsx']={}

conf["datasets"]["sidis"]["xlsx"][9000]="notebooks/9000.xlsx"  #   for EIC simulation 

conf['datasets']['sidis']['norm']={}
for k in conf['datasets']['sidis']['xlsx']: 
    conf['datasets']['sidis']['norm'][k]={'value':1,'fixed':True,'min':0,'max':1} 

## parameters

In [4]:
conf['params']={}
conf['shape']=1

### TMD PDF/FF parameters (from upol analysis)

In [5]:
conf['params']={}
conf['params']['pdf']={}
conf['params']['pdf']['widths0 valence']  = {'value':5.89294556274006398056e-01,'fixed':False,'min':0,'max':1}
conf['params']['pdf']['widths0 sea']      = {'value':6.33443286558464269120e-01,'fixed':False,'min':0,'max':1}
conf['params']['ff']={}
conf['params']['ff']['widths0 pi+ fav']   = {'value':1.15920500644793311729e-01,'fixed':False,'min':0,'max':1}
conf['params']['ff']['widths0 pi+ unfav'] = {'value':1.39782079427820671302e-01,'fixed':False,'min':0,'max':1}

##  Simulation

### initialize fitpack libraries

In [6]:
conf['resman'] = RESMAN()
mcsamp=MCSAMP()

loading sidis data sets 9000
multiprocess setup: ncpus=1 / observable


### collect results from global analysis

MC=mcsamp.get_MC_samples('results/mcdata')

In [7]:
path2results='sidis_upol'
MC=mcsamp.get_MC_samples(path2results+'/mcdata')

runs max likelihoods
166.863761836
166.863761671
166.863761847
166.863761661
166.863761802
166.863761713
166.863761649
166.863761707
166.863761694
166.863761654
166.863761691
166.863761764
166.863761832
166.863761779
166.863761673
166.863761779
166.863761765
166.86376177
166.863761847
166.863761867
sample  size= 6934
sample2 size= 1762


## compute prediction for new data

In [8]:
data={'weights':MC['weights2']}
cnt=0
for s in MC['samples2']:
    cnt+=1    
    lprint('%d/%d'%(cnt,len(MC['samples2'])))  
    conf['resman'].get_residuals(s);
    for k in conf['resman'].sidisres.tabs:
        if k  not in data: data[k]=[]
        thy=conf['resman'].sidisres.tabs[k]['thy']    
        data[k].append(thy)

1762/1762

In [9]:
for k in data: data[k]=np.array(data[k])
thy,dthy={},{}
for k in data:
    if k=='weights': continue
    thy[k]=np.einsum('i,ik->k',data['weights'],data[k])
    dthy[k]=np.einsum('i,ik->k',data['weights'],(data[k]-thy[k])**2)**0.5
for k in thy: 
    conf['resman'].sidisres.tabs[k]['thy']=copy.copy(thy[k])
    conf['resman'].sidisres.tabs[k]['dthy']=copy.copy(dthy[k])

##  prepare xlsx table with the simulated values 

This is the original table 

In [10]:
pd.DataFrame(conf['sidis tabs'][9000])[:10]

,Ebeam,Etarget,N,Q2,Shift,W2,alpha,col,dthy,dy,...,shift,stat_u,target,thy,value,x,y,yh,yp,z
0,20,250,1.0,1.840252,0.0,9293.243337,7.825717e+06,EIC,1.421085e-14,5.024590,...,0.0,7.825717e+06,proton,31.017369,9.391777e+09,0.000198,0.489774,3.870631,8.895221,0.005424
1,20,250,1.0,1.839606,0.0,9337.143486,4.215375e+05,EIC,1.110223e-16,4.503691,...,0.0,4.215375e+05,proton,0.949217,3.179206e+08,0.000197,0.492595,4.396417,8.900109,0.006590
2,20,250,1.0,1.805872,0.0,8939.036093,3.067159e+06,EIC,3.552714e-15,6.864715,...,0.0,3.067159e+06,proton,6.862939,1.442687e+09,0.000202,0.474710,2.001076,8.865791,0.036008
3,20,250,1.0,1.809045,0.0,8999.296404,1.902285e+05,EIC,8.326673e-17,6.107796,...,0.0,1.902285e+05,proton,0.144254,6.474367e+07,0.000201,0.476415,2.763836,8.871631,0.036157
4,20,250,1.0,1.826244,0.0,9039.867206,1.463781e+06,EIC,0.000000e+00,7.552547,...,0.0,1.463781e+06,proton,3.975299,6.571755e+08,0.000202,0.478314,1.318853,8.871400,0.071411
5,20,250,1.0,1.819185,0.0,9049.734026,9.567382e+04,EIC,1.387779e-17,6.784667,...,0.0,9.567382e+04,proton,0.076399,3.275386e+07,0.000201,0.478401,2.089759,8.874426,0.072066
6,20,250,1.0,1.821811,0.0,9017.926094,2.644302e+05,EIC,2.220446e-16,8.497063,...,0.0,2.644302e+05,proton,1.338292,1.286776e+08,0.000202,0.476417,0.373121,8.870185,0.188557
7,20,250,1.0,1.842989,0.0,9122.746501,1.988510e+04,EIC,4.857226e-17,7.753605,...,0.0,1.988510e+04,proton,0.029484,8.489516e+06,0.000202,0.481360,1.122359,8.875963,0.198334
8,20,250,1.0,1.757785,0.0,8701.029724,6.531993e+04,EIC,1.665335e-16,9.454356,...,0.0,6.531993e+04,proton,0.134174,1.046914e+07,0.000202,0.463111,-0.602060,8.852296,0.521360
9,20,250,1.0,1.814343,0.0,8980.963265,5.884394e+03,EIC,1.040834e-17,8.684193,...,0.0,5.884394e+03,proton,0.017982,9.912191e+05,0.000202,0.477066,0.183938,8.868131,0.527503


Next we modify the column "value" of the table. The column represent predictions. You can 
construct by setting: 

tab['value']=tab['thy']  (no statistical noise)

tab['value']=tab['thy'] + np.random(len(tab['thy']))*tab['stat_u']  (with statistical noise)



In [13]:
tab=conf['sidis tabs'][9000]
tab['value']=tab['thy']
tab['stat_u']=tab['relative']*tab['thy']
tab=pd.DataFrame(tab)
tab[:10]

,Ebeam,Etarget,N,Q2,Shift,W2,alpha,col,dthy,dy,...,shift,stat_u,target,thy,value,x,y,yh,yp,z
0,20,250,1.0,1.840252,0.0,9293.243337,7.825717e+06,EIC,1.421085e-14,5.024590,...,0.0,0.025845,proton,31.017369,31.017369,0.000198,0.489774,3.870631,8.895221,0.005424
1,20,250,1.0,1.839606,0.0,9337.143486,4.215375e+05,EIC,1.110223e-16,4.503691,...,0.0,0.001259,proton,0.949217,0.949217,0.000197,0.492595,4.396417,8.900109,0.006590
2,20,250,1.0,1.805872,0.0,8939.036093,3.067159e+06,EIC,3.552714e-15,6.864715,...,0.0,0.014591,proton,6.862939,6.862939,0.000202,0.474710,2.001076,8.865791,0.036008
3,20,250,1.0,1.809045,0.0,8999.296404,1.902285e+05,EIC,8.326673e-17,6.107796,...,0.0,0.000424,proton,0.144254,0.144254,0.000201,0.476415,2.763836,8.871631,0.036157
4,20,250,1.0,1.826244,0.0,9039.867206,1.463781e+06,EIC,0.000000e+00,7.552547,...,0.0,0.008855,proton,3.975299,3.975299,0.000202,0.478314,1.318853,8.871400,0.071411
5,20,250,1.0,1.819185,0.0,9049.734026,9.567382e+04,EIC,1.387779e-17,6.784667,...,0.0,0.000223,proton,0.076399,0.076399,0.000201,0.478401,2.089759,8.874426,0.072066
6,20,250,1.0,1.821811,0.0,9017.926094,2.644302e+05,EIC,2.220446e-16,8.497063,...,0.0,0.002750,proton,1.338292,1.338292,0.000202,0.476417,0.373121,8.870185,0.188557
7,20,250,1.0,1.842989,0.0,9122.746501,1.988510e+04,EIC,4.857226e-17,7.753605,...,0.0,0.000069,proton,0.029484,0.029484,0.000202,0.481360,1.122359,8.875963,0.198334
8,20,250,1.0,1.757785,0.0,8701.029724,6.531993e+04,EIC,1.665335e-16,9.454356,...,0.0,0.000837,proton,0.134174,0.134174,0.000202,0.463111,-0.602060,8.852296,0.521360
9,20,250,1.0,1.814343,0.0,8980.963265,5.884394e+03,EIC,1.040834e-17,8.684193,...,0.0,0.000107,proton,0.017982,0.017982,0.000202,0.477066,0.183938,8.868131,0.527503


### save the xlsx file

In [14]:
data=pd.DataFrame(tab)
writer=pd.ExcelWriter('9000.xlsx')
data.to_excel(writer,'sheet1')
writer.save() 